In [ ]:

#Create the dataset
dataset = EmotionDataset(input_ids, attention_masks, labels)

#DataLoader
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(emotions)
)

optimizer = AdamW(model.parameters(), lr=5e-5)

loss_fn = BCEWithLogitsLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    print(f"Starting epoch {epoch+1}/{epochs}")


    batch_progress = tqdm(enumerate(train_loader), total=len(train_loader), desc="Batch")
    for step, batch in batch_progress:
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['labels'].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, attention_mask=b_input_mask)

        loss = loss_fn(outputs.logits, b_labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        batch_progress.set_postfix({'Training Loss': '{:.3f}'.format(loss.item())})

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} - Average training loss: {avg_train_loss}")

# Save the model
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')